<img src="imperial_logo.png" width="275" align="left">
<p style="text-align: right">
    Created by Dong-Woo (Dom) Ko<br>Email: dk1713@ic.ac.uk<br><a>HTML Version (This will be a link)</a>
</p>
<br>
# 3D Linear Transformation (Sphere)

## Learning Objectives
* To aid in visualisation of 3D linear transformation (Matrix transformation)
* To show that commutation is not obvious in 3D transformation.

## Table of Contents
1. Introduction
2. Common Features
    1. Buttons
    2. Equation
    3. Python Codes
3. Rotation in $I\!R^3$
4. Reflection in $I\!R^3$

## 1. Introduction
We live in 3 Dimensions but that doesn't mean it is a simple matter to visualise 3D transformations. Our aim is to relieve you of that problem by providing you with this visualisation tool. We hope this will help you get a first class.

By plotting a sphere, we are illustrating one of the features of orthogonal linear functions:
     Matrix determinant = 1.
     
This tool is equipped with two 3D orthogonal linear transformations:
1. Rotations and,
2. Reflections

## 2. Common Features

### 2.A. Buttons
Each tab has three buttons
1. [Play] - "plays" the transformation animation,
2. [Undo] - "undos" the applied transformation, and
3. [Reset to (-2,-2,2)] - "resets" the visualisation.

### 2.B. Equation
Shows Matrix equation showing initial $(x_0, y_0, z_0)$ and transformed vectors $(x, y, z)$, and Matrix of transformation.

$$
\begin{pmatrix} x \\ y \\ z \end{pmatrix} = \begin{pmatrix} a & b & c \\ d & e & f \\ g & h & i\end{pmatrix}\begin{pmatrix} x_0 \\ y_0 \\ z_0 \end{pmatrix}
$$

### 2.C. Python Code
To illustrate the this noteboook, python code is written for you to try few transformations in the next chapter and this is preparation for that.<br> Although, do use the html link for more interactive visualisation.

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import numpy as np
import ipywidgets as widgets

In [ ]:
def addOpacity(string, intensity="0.5"):
    strList = list(string)
    strList.pop()
    string = "".join(strList) + "," + intensity +")"
    return string

In [ ]:
def sp2c(r, theta, phi):
    return [
        r * np.sin(theta) * np.cos(phi),
        r * np.sin(theta) * np.sin(phi),
        r * np.cos(theta)
    ]

In [ ]:
def c2sp(x, y, z):
    r = 0
    theta = 0
    phi = 0
    if (x*x + y*y + z*z != 0):
        r = np.sqrt(x*x + y*y + z*z)
        theta = np.arccos(z/r)
        phi = np.arctan2(y,x)
    return [r, theta, phi]

### Objects

In [ ]:
class Line: 
    def __init__(self, pointList):
        self.x = []
        self.y = []
        self.z = []
        
        for i in range(len(pointList)):
            self.x.append(pointList[i][0])
            self.y.append(pointList[i][1])
            self.z.append(pointList[i][2])
        
    def gObject(self, color="rgb(210,64,0)"):
        lineObject = go.Scatter3d(mode="lines",
                                  x=self.x,
                                  y=self.y,
                                  z=self.z,
                                  line=dict(color=(color),
                                            width=7)
                                 )
        return lineObject
    
    def arrowHead(self, color, width=7, wingLen=0.1, dash="solid"):
        lastElm = len(self.x) - 1
        [r, theta, phi] = c2sp(self.x[lastElm]-self.x[0], self.y[lastElm]-self.y[0], self.z[lastElm]-self.z[0])
        offset = [self.x[0], self.y[0], self.z[0]]
        frac = wingLen*r
        sin45 = np.sin(np.pi/4)
        d = r - frac * sin45
        wingLength = np.sqrt(np.power(frac*sin45,2) + d*d)
        wingAngle = np.arccos(d/wingLength)
        
        wings_xyz = [
            np.add(sp2c(wingLength, theta + wingAngle, phi), offset),
            np.add(sp2c(wingLength, theta - wingAngle, phi), offset)
        ]

        wings = {
            "type": "scatter3d",
            "mode": "lines",
            "x": [wings_xyz[0][0], self.x[lastElm], wings_xyz[1][0]],
            "y": [wings_xyz[0][1], self.y[lastElm], wings_xyz[1][1]],
            "z": [wings_xyz[0][2], self.z[lastElm], wings_xyz[1][2]],
            "line": dict(color=color, width=width)
        }

        return wings

In [ ]:
class Sphere:
    def __init__(self, radius=5, center=[0, 0, 0]):
        self.radius = radius
        self.center = center
        meshSize = 20
        theta = np.linspace(0,2*np.pi,meshSize)
        phi = np.linspace(0,np.pi,meshSize)
        self.x = radius*np.outer(np.cos(theta),np.sin(phi)) + center[0]
        self.y = radius*np.outer(np.sin(theta),np.sin(phi)) + center[1]
        self.z = radius*np.outer(np.ones(meshSize),np.cos(phi)) + center[2]
        
    def gObject(self, color=[[0.0, "rgb(0,62,116)"], [1.0, "rgb(255,255,255)"]]):
        sphere = go.Surface(x=self.x.tolist(),
                            y=self.y.tolist(),
                            z=self.z.tolist(),
                            showscale=False,
                            opacity=0.7,
                            colorscale=color
                           )       
        return sphere

### Frames computation:

In [ ]:
def computeFrames(rotation, theta, point, frames):
    vecPoint = np.matrix([[point[0]],
                          [point[1]],
                          [point[2]]
                         ])
    t = np.linspace(0, theta, frames)
    
    lineList = [point]
    output = []
    for i in t:
        newPoint = rotation(i)*vecPoint
        ptList = np.reshape(newPoint,(1,3)).tolist()[0]
        lineList.append(ptList)
        newVec = Line([[0,0,0], ptList])
        output.append(dict(data = [
            newVec.gObject("rgb(0,0,0)"),
            newVec.arrowHead("rgb(0,0,0)"),
            Line(lineList).gObject("rgb(219,209,0)")])
        )
    return output

## 3. Rotation in $I\!R^3$

This tab explores the rotations about the three cartesian coordinate axes. Note that this visualisation will not go over the complete theory of rotation in $I\!R^3$. But, this will help you to see the fundamental difference between rotations in $I\!R^3$ and $I\!R^2$.

### Rotational Matrices:
$$
    R_1(\theta) = \begin{pmatrix} 1 & 0 & 0 \\ 0 & \cos\theta & -\sin\theta \\ 0 & \sin\theta & \cos\theta \end{pmatrix}, \quad
    R_2(\theta) = \begin{pmatrix} \cos\theta & 0 & \sin\theta \\ 0 & 1 & 0 \\ -\sin\theta & 0 & \cos\theta \end{pmatrix}, \quad
    R_3(\theta) = \begin{pmatrix} \cos\theta & -\sin\theta & 0 \\ \sin\theta & \cos\theta & 0 \\ 0 & 0 & 1\end{pmatrix}.
$$ 
&emsp; $\theta $ is the angle of rotation, <br>
&emsp; $R_1(\theta)$ is rotational matrix about x-axis, <br>
&emsp; $R_2(\theta)$ is rotational matrix about y-axis, <br>
&emsp; $R_3(\theta)$ is rotational matrix about z-axis.

### Python Code for rotational Matrices:

In [ ]:
def roXaxis(theta):
    M = np.matrix([[1, 0, 0],
                   [0, np.cos(theta), -np.sin(theta)], 
                   [0, np.sin(theta), np.cos(theta)]
                  ])
    return M

def roYaxis(theta):
    M = np.matrix([[np.cos(theta), 0, np.sin(theta)],
                   [0, 1, 0],
                   [-np.sin(theta), 0, np.cos(theta)]
                  ])
    return M

def roZaxis(theta):
    M = np.matrix([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta), np.cos(theta), 0],
                   [0, 0 ,1]
                  ])
    return M

### Frame Parameters:

In [ ]:
frameSize = 24
initialPoint = [-2., -2., 2.]

In [ ]:
data = []
for i in range(3):
    data.append({"type": "scatter3d"})
        
radius = 2*np.sqrt(3)
sphere = Sphere(radius)
data.append(sphere.gObject())

In [ ]:
layout=dict(width=1000, height=500,
            hovermode='closest',
            margin=dict(l=0,r=0,t=0,b=0),
            updatemenus=[dict(x=-0.05,
                              y=0.15,
                              yanchor="top",
                              xanchor="right",
                              showactive=False,
                              type="buttons",
                              pad={"t": 87, "r": 10},
                              buttons=[dict(method="animate",
                                            args=[None,
                                                  dict(fromcurrent=True,
                                                       transition=dict(duration=50,
                                                                       easing="quadratic-in-out"
                                                                      ),
                                                       frame=dict(duration=50,
                                                                  redraw=False
                                                                 )
                                                      )
                                                 ],
                                            label="Play"
                                           ),
                                       dict(method="animate",
                                           args=[[None],
                                                 dict(mode="immediate",
                                                      transition=dict(duration=0
                                                                     ),
                                                      frame=dict(duration=0,
                                                                 redraw=False
                                                                )
                                                     )
                                                ],
                                            label="Pause"
                                           )
                                      ]
                             )
                        ],
            showlegend=False,
            scene=dict(camera=dict(eye=dict(x=-0.8,y=-1.3,z=1))))

### Transformation:

Try changing "roXaxis" to "roYaxis" below. <br>
Also try changing the values of "theta".

In [ ]:
theta = np.pi/2
frames = computeFrames(roXaxis, theta, initialPoint, frameSize)
figure = dict(data=data, frames=frames, layout=layout)
py.iplot(figure)

Note that there is minor bug with Plotly animation where it will start playing, please ignore that.

Note that each of the rotations in $I\!R^3$ have the same properties as those in $I\!R^2$ except for one. <br>

\begin{align}
    R_1(\theta)R_2(\theta) &= \begin{pmatrix} 1 & 0 & 0 \\ 0 & \cos\theta & -\sin\theta \\ 0 & \sin\theta & \cos\theta \end{pmatrix} \begin{pmatrix} \cos\theta & 0 & \sin\theta \\ 0 & 1 & 0 \\ -\sin\theta & 0 & \cos\theta \end{pmatrix}\\
     &= \begin{pmatrix} \cos\theta & 0 & \sin\theta \\ -\sin^2\theta & \cos\theta & -\sin\theta\cos\theta \\ -\sin\theta\cos\theta & \sin\theta & \cos^2\theta\end{pmatrix},
\end{align}
and compare it to,
\begin{align}
    R_2(\theta)R_1(\theta) &= \begin{pmatrix} \cos\theta & 0 & \sin\theta \\ 0 & 1 & 0 \\ -\sin\theta & 0 & \cos\theta \end{pmatrix} \begin{pmatrix} 1 & 0 & 0 \\ 0 & \cos\theta & -\sin\theta \\ 0 & \sin\theta & \cos\theta \end{pmatrix}\\
     &= \begin{pmatrix} \cos\theta & \sin^2\theta & \sin\theta\cos\theta \\ 0 & \cos\theta & -\sin\theta \\ -\sin\theta & \sin\theta\cos\theta & \cos^2\theta\end{pmatrix}.
\end{align}
and you can see that
$$
    R_1(\theta)R_2(\theta) \neq R_2(\theta)R_1(\theta)
$$

This shows that rotations in $I\!R^3$ is not commutative. This property has important implications for quantum mechanical systems.

### Non-commutation of Rotations in $I\!R^3$

Try below visualisation to see above statement.

In [ ]:
def computeCompositeFrames(rotation1, rotation2, theta, point, frames, color0, color1="rgb(210,64,0)", color2="rgb(210,64,0)"):
    vecPoint = np.matrix([[point[0]],
                          [point[1]],
                          [point[2]]
                         ])
    t = np.linspace(0, theta, frames)
    
    lineList = [point]
    output = []
    for i in t:
        newPoint = rotation1(i)*vecPoint
        ptList = np.reshape(newPoint,(1,3)).tolist()[0]
        lineList.append(ptList)
        newVec = Line([[0,0,0],ptList])
        output.append([newVec.gObject(color0),
                       newVec.arrowHead(color0),
                       Line(lineList).gObject(color1),
                       Line([[0., 0., 0.], [0., 0., 0.]]).gObject()
                      ])
    lineList1 = [ptList]
    for j in t[1:]:
        newPoint2 = rotation2(j)*newPoint
        ptList = np.reshape(newPoint2,(1,3)).tolist()[0]
        lineList1.append(ptList)
        newVec = Line([[0,0,0],ptList])
        output.append([newVec.gObject(color0),
                       newVec.arrowHead(color0),
                       Line(lineList1).gObject(color2),
                       Line(lineList).gObject(addOpacity(color1, "0.7"))
                      ])
        
    return output

In [ ]:
data = []
for i in range(16):
    data.append({"type": "scatter3d"})
        
radius = 2*np.sqrt(3)
sphere = Sphere(radius)
data.append(sphere.gObject())

frames = []
frameList1 = computeCompositeFrames(roXaxis,roYaxis,
                                    np.pi/2,
                                    initialPoint, 
                                    frameSize,
                                    "rgb(0,0,0)",
                                    "rgb(182,109,255)",
                                    "rgb(219,209,0)"
                                   )
frameList2 = computeCompositeFrames(roYaxis,roXaxis,
                                    np.pi/2,
                                    initialPoint, 
                                    frameSize,
                                    "rgb(255,255,255)",
                                    "rgb(219,209,0)",
                                    "rgb(182,109,255)"
                                   )
for i in range(len(frameList1)):
    frames.append(dict(data=[frameList1[i][0],
                             frameList1[i][1],
                             frameList1[i][2],
                             frameList1[i][3],
                             frameList2[i][0],
                             frameList2[i][1],
                             frameList2[i][2],
                             frameList2[i][3],
                            ], 
                       name="frame %i" %i
                      )
                 )

In [ ]:
steps=[]
for i in range(0,frameSize,1):
    step = dict(label='R1',
                method='animate',
                args=[["frame %i" %i],
                      dict(mode="immediate",
                           transition=dict(duration=300
                                          ),
                           frame=dict(duration=300,
                                      redraw=False
                                     )
                          )
                     ]
               )
    steps.append(step)
    
for i in range(0,frameSize-1,1):
    step = dict(label='R2',
                method='animate',
                args=[["frame %i" %(i + frameSize)],
                      dict(mode="immediate",
                           transition=dict(duration=300
                                          ),
                           frame=dict(duration=300,
                                      redraw=False
                                     )
                          )
                     ]
               )
    steps.append(step)

sliders= [dict(
    active=0,
    pad={'t':2*frameSize},
    steps=steps
)]

layout=dict(width=1000, height=500,
            title='Non-Communativeness of 3D Rotations', 
            hovermode='closest',
            margin=dict(l=0,r=0,t=25,b=0),
            updatemenus=[dict(x=-0.05,
                              y=0.15,
                              yanchor="top",
                              xanchor="right",
                              showactive=False,
                              type="buttons",
                              pad={"t": 87, "r": 10},
                              buttons=[dict(method="animate",
                                            args=[None,
                                                  dict(fromcurrent=True,
                                                       transition=dict(duration=50,
                                                                       easing="quadratic-in-out"
                                                                      ),
                                                       frame=dict(duration=50,
                                                                  redraw=False
                                                                 )
                                                      )
                                                 ],
                                            label="Play"
                                           ),
                                       dict(method="animate",
                                           args=[[None],
                                                 dict(mode="immediate",
                                                      transition=dict(duration=0
                                                                     ),
                                                      frame=dict(duration=0,
                                                                 redraw=False
                                                                )
                                                     )
                                                ],
                                            label="Pause"
                                           )
                                      ]
                             )
                        ],
            showlegend=False,#This can be used to isolate individual trace
            sliders=sliders)

In [ ]:
figure = dict(data=data, frames=frames, layout=layout)
py.iplot(figure)

## 4. Reflection in $I\!R^3$

This tab explores reflections on three planes:
1. Reflection on x = 0,
2. Reflection on y = 0, and
3. Reflection on z = 0.